In [ ]:
# 1. Imports and Setup

import pandas as pd
import ast
import sys
import re
import os
sys.path.append(os.path.abspath("../src"))
from collections import Counter
project_root = os.path.abspath("..")
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
data_path = os.path.join(project_root, "data", "reviews_with_sentiment.csv")
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from analysis.visualization import plot_sentiment_trends, plot_rating_distribution, plot_keyword_cloud
df = pd.read_csv(data_path)

from analysis.insights import extract_drivers_painpoints, compare_banks
from analysis.recommendations import generate_recommendations
from analysis.visualization import plot_sentiment_trends, plot_rating_distribution, plot_keyword_cloud
from analysis.ethics import note_biases